In [ ]:
import sys
import cv2
import csv
import numpy as np
#from retinaface import RetinaFace


import import_ipynb


from Image_Utilities import readImage, writeImage, showImage, showImageBox, getImageCrop, showImageMetadata
from Args import yoloArguments

In [ ]:
class YOLO:
    """
    This class implements YOLO algorithm to get the box of a person in a given image
    """
    
    def __init__(self, args):
        """
        YOLO class constructor
        """
        self.args = args
        self.yolo = None
        self.outputs = None
        
        print("Image metadata:")
        showImageMetadata(self.args["image_path"])
        self.build_yolo()
        
    def build_yolo(self):
        """
        This methods build YOLO using OpenCV
        """
        
        self.classes = []
        with open(self.args["cocoNames"], "rt") as f:
            self.classes = f.read().rstrip("\n").split("\n")
        print("Numero de clases: ",len(self.classes))

    
        self.yolo = cv2.dnn.readNet(self.args["yoloWeights"],self.args["yoloCfg"])
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        
    def process_image(self):
        """
        This method process a specific image
        """
        
        self.image = readImage(self.args["image_path"])
        self.height, self.weight, _ = self.image.shape 
        
        blob = cv2.dnn.blobFromImage(self.image, 1/255, (320,320), [0,0,0], 1, crop=False)
        self.yolo.setInput(blob)

        self.outputs = self.yolo.forward([self.yolo.getLayerNames()[i-1] for i in self.yolo.getUnconnectedOutLayers()])  
      
    def detect(self, confidenceThreshold=0.5, nmsThreshold=0.3):
        """
        This mehtod return the coordinates, class names and confident values of the detection on 
        a already processed image 
        """ 
        
        if self.outputs is None:
            print("Output is not initialized. Error from \"detect\" function")
            return 0
        Bbox = [] #x,y,w,h
        classIDs =  []
        confs = []

        self.coords = []
        self.names = []
        self.confidences = []

        for output in self.outputs:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if confidence > confidenceThreshold:
                    w,h = int(detection[2]*self.weight), int(detection[3]*self.height) 
                    x,y = int((detection[0]*self.weight)-w/2), int((detection[1]*self.height)-h/2)
                    Bbox.append([x,y,w,h])
                    classIDs.append(classID)
                    confs.append(float(confidence))

        indices = cv2.dnn.NMSBoxes(Bbox, confs, confidenceThreshold, nmsThreshold)
        
        for i in indices:
            if self.classes[classIDs[i]] == "person":
                self.coords.append(Bbox[i])
                self.names.append(self.classes[classIDs[i]])
                self.confidences.append(confs[i])

    
    def showDetections(self, imageName=None, savePath=None, RP=None, numbers=None):
        """
        This method shows the detection and ask the user if it is correct before saving it
        """
        
        print(len(self.coords))
        print("YOLO detection:")
        
        dataToSave = []
        for c, n, con in zip(self.coords, self.names, self.confidences):
            if imageName is not None and savePath is not None and RP is not None and numbers is not None:
                dorsal = -1
                answer = ""
                if len(self.confidences) == 1 and len(numbers) == 1:
                    answer = "y"
                    dorsal = numbers[0]
                else:
                    showImageBox(self.image, c, n.upper()+str(int(con*100))+"%")
                
                
                while not (answer == "q" or answer == "y" or answer =="n" or answer =="c" or answer =="s"):
                    answer = input("Is this detection right?(y/n)")
                if answer == "q":
                    sys.exit()
                if answer == "s":
                    break
                if answer == "y" or answer == "n" or answer == "c":
                    if answer == "y" or answer == "c":
                        if dorsal == -1:
                            dorsal = input("Dorsal number?")
                        dataToSave.append([imageName, dorsal, c[0], c[1], c[2], c[3], RP])
                    continue
                    
        if dataToSave != []:
            import csv
            with open(savePath, "a") as f:
                print(savePath)
                writer = csv.writer(f)
                for d in dataToSave:
                    print(d)
                    writer.writerow(d)

    def getCoords(self):
        """
        This method return the current coords
        """
        return self.coords
    
    def getImage(self):
        """
        This method return the current image
        """
        return self.image
        
    def setCoords(self, coords):
        """
        This method set the current coords
        """
        self.coords = coords
        
    def setImage(self, image_path):
        """
        This method set the current image path
        """
        self.args['image_path'] = image_path
